In [13]:
import json

# 📌 1️⃣ 두 개의 JSON 파일 로드
with open("unlabeled_books_with_summary_edit.json", "r", encoding="utf-8") as f:
    unlabeled_books = json.load(f)

with open("labeling_novels_filtered.json", "r", encoding="utf-8") as f:
    labeled_books = json.load(f)

# ✅ 2️⃣ 중복 방지: 책 제목 기준으로 딕셔너리 생성
merged_books = {book["title"]: book for book in unlabeled_books}  # unlabeled 데이터 추가

# ✅ 3️⃣ labeled_books 추가 (중복 방지)
for book in labeled_books:
    title = book["title"]
    if title not in merged_books:
        merged_books[title] = book  # 기존에 없으면 추가

# 📌 4️⃣ 최종 JSON 파일로 저장
output_file = "merged_books.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(list(merged_books.values()), f, ensure_ascii=False, indent=4)

print(f"✅ 두 개의 JSON 파일이 성공적으로 병합되었습니다! ('{output_file}'에 저장됨)")


✅ 두 개의 JSON 파일이 성공적으로 병합되었습니다! ('merged_books.json'에 저장됨)


In [10]:
import json

# 📌 1️⃣ 병합된 JSON 파일 로드
with open("merged_books.json", "r", encoding="utf-8") as f:
    merged_books = json.load(f)

# ✅ 2️⃣ "summary" 값이 "줄거리 없음"이 아닌 데이터만 필터링
filtered_books = [book for book in merged_books if book["summary"].strip() != "줄거리 없음"]

# 📌 3️⃣ 새로운 JSON 파일로 저장
output_file = "merged_books_filtered.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(filtered_books, f, ensure_ascii=False, indent=4)

print(f"✅ '줄거리 없음' 데이터가 제거된 파일이 '{output_file}'에 저장되었습니다!")


✅ '줄거리 없음' 데이터가 제거된 파일이 'merged_books_filtered.json'에 저장되었습니다!


In [ ]:
import json
import faiss
from sentence_transformers import SentenceTransformer

# ✅ 문장 임베딩 모델 로드 (KoSimCSE-roberta)
embedding_model = SentenceTransformer("BM-K/KoSimCSE-roberta")

# ✅ 책 데이터 로드
with open("merged_books_filtered.json", "r", encoding="utf-8") as f:
    books_data = json.load(f)

# ✅ 책 요약을 임베딩 변환
book_summaries = [book["summary"] for book in books_data]
book_titles = [book["title"] for book in books_data]

# 🔹 임베딩 변환
book_embeddings = embedding_model.encode(book_summaries, convert_to_numpy=True)

# 🔹 FAISS 인덱스 생성 (유사도 검색용)
dimension = book_embeddings.shape[1]  # 벡터 차원 (768)
faiss_index = faiss.IndexFlatL2(dimension)  # L2 거리 기반 검색
faiss_index.add(book_embeddings)  # 벡터 추가

# ✅ FAISS 인덱스 저장
faiss.write_index(faiss_index, "book_faiss_index.bin")

print("✅ 책 요약 데이터가 FAISS 벡터 DB에 저장되었습니다.")


No sentence-transformers model found with name BM-K/KoSimCSE-roberta. Creating a new one with mean pooling.


✅ 책 요약 데이터가 FAISS 벡터 DB에 저장되었습니다.


In [21]:
def find_similar_books(user_story, top_k=5):
    """유저가 입력한 스토리에 가장 유사한 책을 추천"""
    # 🔹 입력된 스토리를 문장 임베딩 변환
    user_embedding = embedding_model.encode([user_story], convert_to_numpy=True)

    # 🔹 FAISS를 이용한 유사한 책 검색
    distances, indices = faiss_index.search(user_embedding, top_k)

    # 🔹 검색된 책 목록 출력
    recommended_books = []
    for i in range(top_k):
        idx = indices[0][i]  # 검색된 책 인덱스
        title = book_titles[idx]  # 책 제목
        description = books_data[idx]["description"]  # 📌 원본 설명 가져오기
        recommended_books.append(f"📖 제목: {title}\n📌 줄거리: {description}\n")

    return "\n".join(recommended_books)

# ✅ 테스트: 유저가 입력한 스토리를 바탕으로 책 추천
user_story = "결혼을 앞둔 약혼녀가 사라져서 그녀를 추적하는 소설."
print(find_similar_books(user_story, top_k=5))

📖 제목: 오만과 선량
📌 줄거리: 가케루는 오랜 시간 만난 연인과 결혼이라는 결론에 다다르지 못하고 헤어졌다.
 오랜 결혼 활동 끝에 결혼 정보 앱에서 착하고 성실한 마미를 만난다.
 가케루는 마미와 결혼해야겠다는 확신은 없지만 결국 2년 동안의 연애 끝에 결혼을 약속한다.
 다니던 회사의 송별회 다음날, 마미는 약혼 반지를 비롯해 많은 것을 그대로 남겨둔 채 홀연히 사라진다.
 단서는 마미가 줄곧 말해 왔던 스토커의 존재.
 가케루는 스토커가 있다는 마미의 고향 군마로 향해 마미를 둘러싼 사람들을 만나며 마미의 과거를 파헤친다.


📖 제목: 원도
📌 줄거리: “나는 왜 살아 있는가.
 이것이 아니다.

 나는 왜 죽지 않았는가.
 이것이다”

살아내기 위해 응시해야 하는 내 속의 광활한 구멍 하나 
이 사람 ‘원도’와 다르지 않은 우리를 위한 
삶과 구원에 대한 통렬한 이야기

2006년 〈실천문학〉으로 작품 활동을 시작한 이래 2010년 첫 장편소설인 《당신 옆을 스쳐간 그 소녀의 이름은》으로 한겨레문학상을 수상하며 본격적인 이름을 알린 지 십수 년 남짓.
 처연한 비관의 세계에서 시작한 그는 2023년 이상문학상을 받으며 “등단 이후 십여 년간 한결같은 걸음걸이로 걸어온 작가의 작품 세계가 마침내 새로운 경지로 들어섰음을 보여준다.
 눈이 부시다”(소설가 윤대녕)라는 평을 받았다.
 같은 해 출간한 장편소설 《단 한 사람》을 통해서는 십여 년간 곱씹은 질문에 마침내 마침표를 찍으며 작가적 전환점을 맞기도 했다.
 지구에서 가장 키가 크고 오래 사는 생물, 수천 년 무성한 나무의 수명 가운데 이파리 한 장만큼을 빌려 죽을 위기에 처한 단 한 명만 살릴 수 있는, ‘나무와 인간 사이 수명 중개인’의 이야기인 《단 한 사람》은 출간되자마자 쇄를 거듭하며 하반기 최대 베스트셀러로 등극했음은 물론, 〈한겨레21〉 〈시사인〉 〈채널예스〉에서 올해의 책으로 선정되었다.
 
20여만 부가 판매되며 역주행 열풍을 이끈 《구의 증명》에서부터 소설적 성취의 완결을 보여준

In [17]:
# 🔹 FAISS 인덱스 내의 데이터 수 확인
print(f"FAISS 인덱스 크기: {faiss_index.ntotal}")  # 🔹 저장된 벡터 개수 확인

# 🔹 books_data의 개수 확인
print(f"책 데이터 개수: {len(book_summaries)}")


FAISS 인덱스 크기: 1323
책 데이터 개수: 1323
